# **Install dan Setting Library**

- Setting selenium khusus Google Colab: https://nariyoo.com/python-how-to-run-selenium-in-google-colab/
- Tidak perlu run kode cell ini jika melakukan running di Jupyter Notebook atau local Python lain

In [2]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

SyntaxError: invalid decimal literal (4230610949.py, line 3)

In [3]:
!pip3 install chromedriver-autoinstaller

  Obtaining dependency information for chromedriver-autoinstaller from https://files.pythonhosted.org/packages/a5/b5/36f0b0add145c371b5282e881a687601899f2d27fae5d0595bc02026b67c/chromedriver_autoinstaller-0.6.4-py3-none-any.whl.metadata


In [4]:
!pip3 install dateparser

  Obtaining dependency information for dateparser from https://files.pythonhosted.org/packages/a4/29/db12aa4dda81580be1999824a689bd52aa40061fc12c9ccdc3feab5ea718/dateparser-1.2.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzlocal from https://files.pythonhosted.org/packages/97/3f/c4c51c55ff8487f2e6d0e618dba917e3c3ee2caae6cf0fbb59c9b1876f2e/tzlocal-5.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/295.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/295.0 kB 660.6 kB/s eta 0:00:01
   ----- --------------------------------- 41.0/295.0 kB 653.6 kB/s eta 0:00:01
   ---------------- ----------------------- 122.9/295.0 kB 1.2 MB/s eta 0:00:01
   ------------------------------ --------- 225.3/295.0 kB 1.3 MB/s eta 0:00:01
   -------------------------------------- - 286.7/295.0 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 295.0/295.0 kB 1.3 MB/s eta 0:00:00


In [5]:
# Import library
import re
import csv
import sys
import time
import dateparser
import pandas as pd
import chromedriver_autoinstaller
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [6]:
# Setting chrome driver
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()

'C:\\Users\\User\\anaconda3\\Lib\\site-packages\\chromedriver_autoinstaller\\128\\chromedriver.exe'

# **Scraping Link Artikel di Google News**

In [7]:
# Set untuk menyimpan daftar link berita
link_set = set()

In [8]:
def split_google_news_link(url):
  url_split = re.split(r"start=(\d+)&sa=", url)
  return url_split[0] + 'start=', '&sa=' + url_split[-1]

In [9]:
def get_google_main_url(url):
  return re.findall(r"^(.*?)(?=&)", url)[0]

In [10]:
def scrape_google_news_link(result_set, number, url):

  url1, url2 = split_google_news_link(url)
  main_url = get_google_main_url(url1)
  print(main_url)

  driver = webdriver.Chrome(options=chrome_options)
  article_xpath = "//div[contains(@class, 'MjjYud')]/div"

  try:
      # Scrape link dari URL di atas, sejumlah <number> artikel (1 page = 10 artikel)
      for i in range(0, number, 10):
          page = url1 + str(i) + url2
          driver.get(page)
          time.sleep(2)

          articles = driver.find_element(By.XPATH, article_xpath)
          articles = articles.find_elements(By.CLASS_NAME, "SoaBEf")
          print(f"Scraping article {i+1} to {i+10}")

          for article in articles:
              a = article.find_element(By.TAG_NAME, "a").get_attribute("href")
              date = dateparser.parse(
                  date_string = article.find_element(By.CLASS_NAME, "OSrXXb").text,
                  languages=['en', 'id'])
              # datetime to string https://www.geeksforgeeks.org/python-strftime-function/
              result_set.add(tuple([a, date.strftime('%Y-%m-%d')]))

  except NoSuchElementException as e:
      # Jika page telah habis
      print('The pages of the web have run out')

  finally:
      driver.close()
      print(f"Successfully got a total of {len(link_set)} unique article links")
      print('----------------------------------------------------------')

Advance google search help: https://www.acgov.org/searchhelp.htm

In [11]:
# Link Google News yang akan di scraping
# Layanan BPJS
scrape_google_news_link(link_set, 1000, 'https://www.google.com/search?q=layanan+bpjs&sca_esv=5eda802b0a6b34cc&sca_upv=1&biw=1280&bih=585&tbm=nws&sxsrf=ADLYWIJod28VXM_sQw-VB0xJlm1jHcLHCA%3A1726464221763&ei=3cDnZqOpLuaRseMPrKr0-Qg&ved=0ahUKEwjji5-13MaIAxXmSGwGHSwVPY8Q4dUDCA0&uact=5&oq=layanan+bpjs&gs_lp=1.25')
# Prolanis BPJS
scrape_google_news_link(link_set, 1000, 'https://www.google.com/search?q=prolanis+bpjs&sca_esv=5eda802b0a6b34cc&sca_upv=1&biw=1280&bih=585&tbm=nws&sxsrf=ADLYWIIUUpaIQD8KRjQsGuC9B8sZUyjzmw%3A1726464600446&ei=WMLnZsH4GvS94-EPjbWz6AY&ved=0ahUKEwjBhOjp3caIAxX03jgGHY3aDG0Q4dUDCA0&uact=5&oq=prolanis+bpjs&gs_lp=1.25')
# BPJS JKN
scrape_google_news_link(link_set, 1000, 'https://www.google.com/search?q=bpjs+jkn&sca_esv=5eda802b0a6b34cc&sca_upv=1&biw=1280&bih=585&tbm=nws&sxsrf=ADLYWILIMAVXdrd4caEexNLdObSIwKF50w%3A1726464687468&ei=r8LnZoCTHM2RseMP5PaOoQ4&ved=0ahUKEwjApqeT3saIAxXNSGwGHWS7I-QQ4dUDCA0&uact=5&oq=bpjs+jkn&gs_lp=1.25')
# BPJS Kesehatan
scrape_google_news_link(link_set, 1000, 'https://www.google.com/search?q=bpjs+kesehatan&sca_esv=5eda802b0a6b34cc&sca_upv=1&biw=1280&bih=585&tbm=nws&sxsrf=ADLYWIJ-SUcjIGJW6GjNo7FO_0sRar0lwg%3A1726464689877&ei=scLnZuGlNY6VseMPgfvb8AI&ved=0ahUKEwihwrqU3saIAxWOSmwGHYH9Fi4Q4dUDCA0&uact=5&oq=bpjs+kesehatan&gs_lp=1.25')

https://www.google.com/search?q=layanan+bpjs
Scraping article 1 to 10
Scraping article 11 to 20
Scraping article 21 to 30
Scraping article 31 to 40
Scraping article 41 to 50
Scraping article 51 to 60
Scraping article 61 to 70
Scraping article 71 to 80
Scraping article 81 to 90
Scraping article 91 to 100
Scraping article 101 to 110
Scraping article 111 to 120
Scraping article 121 to 130
Scraping article 131 to 140
Scraping article 141 to 150
Scraping article 151 to 160
Scraping article 161 to 170
Scraping article 171 to 180
Scraping article 181 to 190
Scraping article 191 to 200
Scraping article 201 to 210
Scraping article 211 to 220
Scraping article 221 to 230
Scraping article 231 to 240
Scraping article 241 to 250
Scraping article 251 to 260
Scraping article 261 to 270
Scraping article 271 to 280
Scraping article 281 to 290
Scraping article 291 to 300
Scraping article 301 to 310
Scraping article 311 to 320
Scraping article 321 to 330
Scraping article 331 to 340
Scraping article 341 to

Scraping article 831 to 840
Scraping article 841 to 850
Scraping article 851 to 860
Scraping article 861 to 870
Scraping article 871 to 880
Scraping article 881 to 890
Scraping article 891 to 900
Scraping article 901 to 910
Scraping article 911 to 920
Scraping article 921 to 930
Scraping article 931 to 940
Scraping article 941 to 950
Scraping article 951 to 960
Scraping article 961 to 970
Scraping article 971 to 980
Scraping article 981 to 990
Scraping article 991 to 1000
Successfully got a total of 22 unique article links
----------------------------------------------------------
https://www.google.com/search?q=bpjs+kesehatan
Scraping article 1 to 10
Scraping article 11 to 20
Scraping article 21 to 30
Scraping article 31 to 40
Scraping article 41 to 50
Scraping article 51 to 60
Scraping article 61 to 70
Scraping article 71 to 80
Scraping article 81 to 90
Scraping article 91 to 100
Scraping article 101 to 110
Scraping article 111 to 120
Scraping article 121 to 130
Scraping article 131 

In [12]:
# Contoh isi link_set
for i, data in enumerate(link_set):
  if (i == 5):
    break
  print(data)

('https://www.unila.ac.id/prolanis-di-klinik-unila-kolaborasi-dengan-kimia-farma/', '2024-05-29')
('https://manado.antaranews.com/berita/261447/rasa-kagum-jabir-yang-terlindungi-kesehatannya-oleh-program-jkn', '2024-09-14')
('https://upk.kemkes.go.id/new/klub-prolanis-upk-berteman-dengan-hipertensi-dan-diabetes-melitus', '2022-08-16')
('https://jateng.tribunnews.com/2024/09/16/inilah-cara-klaim-kacamata-gratis-melalui-bpjs-kesehatan-simak-ketentuannya', '2024-09-16')
('https://belitung.tribunnews.com/2024/09/15/pasangan-bekawan-janjikan-bayar-tunggakan-bpjs-kesehatan-masyarakat-belitung-timur', '2024-09-15')


# **Ubah Set Menjadi DataFrame**

In [13]:
link_df = pd.DataFrame(data=link_set, columns=['url', 'date'])

In [14]:
# Konversi tanggal menjadi datetime, urutkan dari terbaru ke terlama
link_df['date'] = pd.to_datetime(link_df['date'], format='%Y-%m-%d')
link_df = link_df.sort_values(by='date', ascending=False, ignore_index=True)

In [15]:
link_df

,url,date
0,https://rbtv.disway.id/read/66500/angsuran-bpj...,2024-09-16
1,https://jateng.tribunnews.com/2024/09/16/inila...,2024-09-16
2,https://www.rri.co.id/mataram/kesehatan/975069...,2024-09-15
3,https://belitung.tribunnews.com/2024/09/15/pas...,2024-09-15
4,https://kaltimpost.jawapos.com/balikpapan/2385...,2024-09-15
5,https://www.kompas.com/tren/read/2024/09/14/17...,2024-09-15
6,https://www.rri.co.id/daerah/973709/pemkab-mad...,2024-09-15
7,https://www.beritasatu.com/network/suaraindone...,2024-09-15
8,https://toraja.tribunnews.com/2024/09/15/siner...,2024-09-15
9,https://www.kompas.tv/regional/538445/bersilat...,2024-09-15


# **Simpan Link Artikel ke CSV**

In [16]:
output_file = 'daftar_link_berita_bpjs.csv'
link_df.to_csv(output_file, index=False)

message = f"Successfully output the results to the {output_file} file"
print('-' * len(message))
print(message)
print(f"Successfully got {len(link_set)} unique article links")

-----------------------------------------------------------------------
Successfully output the results to the daftar_link_berita_bpjs.csv file
Successfully got 30 unique article links
